In [1]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

In [272]:
%reload_ext autoreload
%autoreload 2

In [273]:
from restaurant_setup import RestaurantSetup

res = RestaurantSetup("https://www.yelp.com/search?find_desc=Restaurants&find_loc=San%20Francisco%2C%20CA&ns=1")
pages = res.get_pag_num()
restaurant = res.create_mongodb()
print(pages)

161


In [8]:
restaurant

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'yelp'), 'restaurants')

In [11]:
li = res.get_restaurant_list(pages)

/Users/yuewengmak/Documents/galvanize/capstone3/restaurant-recommender/restaurant_setup.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /Users/yuewengmak/Documents/galvanize/capstone3/restaurant-recommender/restaurant_setup.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  container_div = rsoup.findAll("div", {"class": "container__373c0__ZB8u4"})


In [13]:
li

[{'title': 'Atlas Tap Room',
  'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/Z9FNtPCMIukYQk52un5NuA/ls.jpg',
  'url': 'https://www.yelp.com/adredir?ad_business_id=u18oQc96cWsK2BtbSgSsgg&campaign_id=i0tJLg-Uu7Sa7p8h4HmMbg&click_origin=search_results&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fatlas-tap-room-san-francisco&request_id=a6469be3e2ac31a5&signature=90f46fddf30636b4755611fa16e8bef701e7d275934aaedf1da2783035ce37cc&slot=0',
  'price': 2,
  'stars': 4.0,
  'reviewCount': 186,
  'phone': '(415) 308-5806',
  'address': '606 Mission St',
  'neighborhood': 'Financial District',
  'cuisines': 'Beer Bar, American (New)'},
 {'title': 'Fog Harbor Fish House',
  'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/Lv7tIRX3Bim9bKqtWifmNg/ls.jpg',
  'url': 'https://www.yelp.com/biz/fog-harbor-fish-house-san-francisco-2?osq=Restaurants',
  'price': 2,
  'stars': 4.0,
  'reviewCount': 6006,
  'phone': '(415) 421-2442',
  'address': 'Pier 39',
  'neighborhood

In [102]:
len(li)

1023

In [306]:
# stop at id 85, continue at id 86
new_li = li[642:655]
len(new_li)

13

In [ ]:
res.split_chunks(new_li, restaurant, 13)

Start New List beginning with 0
Insert 0 item
Inserting...
Insert 1 item
Inserting...
Insert 2 item
Inserting...
Insert 3 item
Inserting...
Insert 4 item
Inserting...
Insert 5 item
Inserting...
Insert 6 item
Inserting...
Insert 7 item
Inserting...
Insert 8 item


In [180]:
import time
from restaurant_page_setup import RestaurantPageSetup

new_att = ['city', 'state', 'zipcode', 'health_score', 'ambience', 
              'reservations', 'take_out', 'apple_pay', 'bike_parking',
              'happy_hour', 'wifi', 'dogs', 'drive_thru', 'vegan_options', 'delivery',
              'credit_cards', 'google_pay', 'parking', 'kids', 
              'outdoor_seating', 'tv', 'waiter_service', 'caters',
               'smoking', 'dancing', 'wheelchair', 'coat',
              'meal', 'groups', 'happy_hour_sp', 'noise_level', 
              'alcohol', 'parking']
boolean_values = [{'reservations': 'Reservations',
                    'take_out': 'Take-out',
                    'wifi': 'Wi-Fi',
                    'apple_pay': 'Apple Pay',
                    'bike_parking': 'Bike Parking',
                    'happy_hour': 'Good For Happy Hour',
                    'dogs': 'Dogs Allowed',
                    'drive_thru': 'Drive-Thru',
                    'vegan_options': 'Vegan',
                    'delivery': 'Delivery',
                    'credit_cards': 'Credit Cards',
                    'google_pay': 'Google Pay',
                    'tv': 'TV',
                    'waiter_service': 'Waiter',
                    'caters': 'Caters',
                    'smoking': 'Smoking',
                    'dancing': 'Dancing',
                    'wheelchair': 'Wheelchair',
                    'coat': 'Coat',
                    'groups': 'Good for Groups',
                    'outdoor_seating': 'Outdoor Seating',
                    'happy_hour_sp': 'Happy Hour Specials' }]
for idx, item in enumerate(li):
    for att in new_att:
        item[att] = None
    rps = RestaurantPageSetup('https://www.yelp.com/biz/abrazo-san-francisco-2?osq=Restaurants')
    rps.setup_driver()
    rps.driver.set_window_position(0,-1000)
    location = rps.driver.find_elements_by_tag_name('address')[-1].find_elements_by_tag_name('p')[-1].text
    if location:
        item['city'] = location.split(",")[0]
        item['state'] = location.split(", ")[1].split(" ")[0]
        item['zipcode'] = location.split(", ")[1].split(" ")[-1]
    attribute_button = rps.driver.find_elements_by_xpath("//p[contains(text(),'Attributes')]")
    if attribute_button:
        attribute_button[0].click()
        ambience_div = rps.driver.find_elements_by_xpath("//span[contains(text(),'Ambience')]")
        health_div = rps.driver.find_elements_by_xpath("//a[contains(text(),'Health Score')]")
        meal_div = rps.driver.find_elements_by_xpath("//span[contains(text(),'Good For')]")
        parking_div = rps.driver.find_elements_by_xpath("//span[(contains(text(),'Parking')) and (not(contains(text(), 'Bike')))]")
        noise = rps.driver.find_elements_by_xpath("//span[contains(text(),'Noise Level')]")
        alcohol = rps.driver.find_elements_by_xpath("//span[contains(text(),'Alcohol')]")

        if ambience_div and ambience_div[0].find_elements_by_xpath("./following-sibling::span"):
            item['ambience'] = ambience_div[0].find_element_by_xpath("./following-sibling::span").text.strip()
        if meal_div and meal_div[0].find_elements_by_xpath("./following-sibling::span"):
            item['meal'] = meal_div[0].find_element_by_xpath("./following-sibling::span").text.strip()
        if health_div and health_div[0].find_elements_by_xpath("../following-sibling::span"):
            item['health_score'] = int(health_div[0].find_element_by_xpath("../following-sibling::span").text.split("out of")[0])
        if noise and noise[0].find_elements_by_xpath("./following-sibling::span"):
            item['noise_level'] = noise[0].find_element_by_xpath("./following-sibling::span").text.strip()
        if alcohol and alcohol[0].find_elements_by_xpath("./following-sibling::span"):
            item['alcohol'] = alcohol[0].find_element_by_xpath("./following-sibling::span").text.strip()
        if parking_div and parking_div[0].find_elements_by_xpath("./following-sibling::span"):
            item['parking'] = parking_div[0].find_element_by_xpath("./following-sibling::span").text.strip()

        for value in boolean_values:
            for k, v in value.items():
                bdiv = rps.driver.find_elements_by_xpath("//span[contains(text(),'" + v + "')]")
                if bdiv and bdiv[0].find_elements_by_xpath("./following-sibling::span"):
                    print(v)
                    item[k] = convert_yes_no_to_boolean(bdiv[0].find_elements_by_xpath("./following-sibling::span")[0].text.strip())
    print(item)
    break
#     if idx == 20:
#         break
# rps = RestaurantPageSetup('https://www.yelp.com/biz/pescatore-san-francisco-2?osq=Restaurants')
# rps.setup_driver()

Reservations
Take-out
Wi-Fi
Bike Parking
Good For Happy Hour
Dogs Allowed
Delivery
Credit Cards
TV
Waiter
Caters
Wheelchair
Good for Groups
Outdoor Seating
{'title': 'Atlas Tap Room', 'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/Z9FNtPCMIukYQk52un5NuA/ls.jpg', 'url': 'https://www.yelp.com/adredir?ad_business_id=u18oQc96cWsK2BtbSgSsgg&campaign_id=i0tJLg-Uu7Sa7p8h4HmMbg&click_origin=search_results&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fatlas-tap-room-san-francisco&request_id=a6469be3e2ac31a5&signature=90f46fddf30636b4755611fa16e8bef701e7d275934aaedf1da2783035ce37cc&slot=0', 'price': 2, 'stars': 4.0, 'reviewCount': 186, 'phone': '(415) 308-5806', 'address': '606 Mission St', 'neighborhood': 'Financial District', 'cuisines': 'Beer Bar, American (New)', 'city': 'San Francisco', 'state': 'CA', 'zipcode': '94109', 'health_score': None, 'ambience': 'Classy', 'reservations': 1, 'take_out': 1, 'apple_pay': None, 'bike_parking': 0, 'happy_hour': 1, 'wifi':

In [52]:
def convert_yes_no_to_boolean(string):
    if string == 'Yes':
        return 1
    else:
        return 0
# groups_div.find_element_by_xpath("./following-sibling::span").text.strip()

In [ ]:
element = rps.driver.find_element_by_xpath("//p[contains(text(),'Attributes')]")

In [ ]:
element.click()

In [ ]:
outdoor_seating = rps.driver.find_element_by_xpath("//span[contains(text(),'Outdoor Seating')]")
outdoor_seating

In [ ]:
outdoor_seating.find_element_by_xpath("./following-sibling::span").text.strip()

In [ ]:
import requests

test_yelp = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=San%20Francisco%2C%20CA&ns=1"
r = requests.get(test_yelp, headers={'user-agent': 'Mozilla/5.0'})

soup = BeautifulSoup(r.content, "html")
# print(soup)
# list_div = soup.find("ul", {"class": "list__373c0__2G8oH"})
pag_div = soup.find("div", {"class": "pagination-links-container__373c0__1vHLX"})
sib_div = pag_div.find_next_sibling()
pag_num = int(sib_div.text.split("of")[-1])

li = []
for p in range(pag_num):
    start = 30*p
    url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=San%20Francisco%2C%20CA&ns=1&start=" + str(start)
    r_url = requests.get(test_yelp, headers={'user-agent': 'Mozilla/5.0'})
    rsoup = BeautifulSoup(r_url.content, "html")
    container_div = rsoup.findAll("div", {"class": "container__373c0__ZB8u4"})
    print(len(container_div))
    
    for container in container_div:
#         print(container.find("div", {"class": "lemon--div__373c0__1mboc"}))
        alink = container.find("a", {"class": "lemon--a__373c0__IEZFH"})
        print(alink)
        break
    break

In [ ]:
restaurant_list = []
for container in container_div:
    ind_rest = {}
#     print(container.find("div", {"class": "lemon--div__373c0__1mboc"}))
    alink = container.find("a", {"class": "lemon--a__373c0__IEZFH link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE"})
    info_div = container.find("div", {"class": "lemon--div__373c0__1mboc container__373c0__39jSv padding-l2__373c0__1Dr82 border-color--default__373c0__3-ifU text-align--right__373c0__1XDu3"})
    phone_div = info_div.findAll("p", {"class": "lemon--p__373c0__3Qnnj text__373c0__2Kxyz text-color--black-extra-light__373c0__2OyzO text-align--right__373c0__1f0KI text-size--small__373c0__3NVWO"})[0]
    address_div = info_div.find("span", {"class": "lemon--span__373c0__3997G raw__373c0__3rcx7"})
    neighborhood_div = info_div.findAll("p", {"class": "lemon--p__373c0__3Qnnj text__373c0__2Kxyz text-color--black-extra-light__373c0__2OyzO text-align--right__373c0__1f0KI text-size--small__373c0__3NVWO"})[-1]
    stars_div = container.find("div", {"class": "i-stars__373c0__1T6rz"})
    review_div = container.find("span", {"class": "reviewCount__373c0__2r4xT"})
    price_div = container.find("span", {"class": "priceRange__373c0__2DY87"})
    cuisine_div = container.findAll("a", {"class": "lemon--a__373c0__IEZFH link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--default__373c0__7tls6"})
    cuisines = []
    
    for c in cuisine_div:
        cuisines.append(c.text)
    ind_rest['title'] = alink.text
    ind_rest['url'] = "https://www.yelp.com" + alink['href']
    if price_div:
        ind_rest['price'] = len(price_div.text)
    else:
        ind_rest['price'] = None
    ind_rest['stars'] = float(stars_div['aria-label'].split("star")[0])
    ind_rest['reviewCount'] = int(review_div.text)
    if phone_div.text != address_div.text:
        ind_rest['phone'] = phone_div.text
    else:
        ind_rest['phone'] = None
    ind_rest['address'] = address_div.text
    ind_rest['neighborhood'] = neighborhood_div.text
    ind_rest['cuisines'] = ", ".join(cuisines)
    print(ind_rest)

In [ ]:
import requests

# test_wine_url = "https://www.vivino.com/quady-winery-electra-red-california-moscato/w/14175?year=2016&price_id=14978727"
# r = requests.get(test_wine_url, headers={'user-agent': 'Mozilla/5.0'})

# soup = BeautifulSoup(r.content, "html")
# print(soup)
# price_div = soup.find("span", {"class": "purchaseAvailability__currentPrice--3mO4u"})
# print(price_div)

def get_variables(url):
    price = 0.0
    vintage = ""
    review = ""
    test_wine_url = url
    test_wine = WineSetup(test_wine_url)
    test_wine.setup_driver()
    run_sel_lazyloading_test(test_wine.driver)

    test_html = BeautifulSoup(test_wine.driver.page_source, 'lxml')
    price_div = test_html.find("span", {"class": "purchaseAvailability__currentPrice--3mO4u"})
    vintage_div = test_html.find("span", {"class": "purchaseAvailability__year--3huqD"})
    review_div = test_html.find("div", {"class": "communityReview__textSection--vu-i-"})
    if price_div:
        price = float(price_div.text.split("$")[1])
    if vintage_div:
        vintage = vintage_div.text
    if review_div:
        review = review_div.find("p").text
    
    return price, vintage, review


In [ ]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
wine_db = client['wine_db']
wines = wine_db['wines']

test = wine_db.wines.find({'price': 0.0, 'type': 'Port'})
count = 0
for t in test:
    count += 1
    price, vintage, review = get_variables(t['url'])
    
    print('-----------------')
    print('Price: ', price)
    print('Vintage: ', vintage)
    print('Review: ', review)
    
print(count)

In [ ]:
# test_list = range(0, 100)
n = 10
for i in range(0, len(li), n):
    print(li[i:i + n])